In [109]:
import pandas as pd
import numpy as np
import glob
from collections import Counter
from nltk.corpus import stopwords

import os, sys, re, json, time, unittest
import itertools, collections

from scipy import stats

import nltk

# Helper libraries for this notebook
from shared_lib import utils, vocabulary

## Data Load

In [3]:
!head 'data_Las_Vegas/2017-10-02.csv'

username;date;retweets;favorites;text;geo;mentions;hashtags;id;permalink
fxm20193;2017-10-02 16:59;0;0;"Pray for vegas #prayforvegas #肉肉美國行 @The Linq, Las Vegas Highest… https://www. instagram.com/p/BZw9OYkB9W_r gE0Ai4SQFydfX5YJm28XsAABUU0/ …";;@The;#prayforvegas #;"915003391880912901";https://twitter.com/fxm20193/status/915003391880912901
krichardsonwi;2017-10-02 16:58;0;0;"I'm at @GoldSpikeDtlv in Las Vegas, NV https://www. swarmapp.com/c/0j9w1BpKJPf";;@GoldSpikeDtlv;;"915003111827271680";https://twitter.com/krichardsonwi/status/915003111827271680
RalphPaglia;2017-10-02 16:58;0;0;"Don’t let Steve Paddock win... visit #LasVegas soon... Celebrate our… https://www. instagram.com/p/BZw9D05gNXej h2Cu7amhc6pjqHPQXLCAg5fSYA0/ …";;;#LasVegas;"915003023180496896";https://twitter.com/RalphPaglia/status/915003023180496896
JLangelerNews;2017-10-02 16:56;1;2;"Vigil at city hall #vegasstrong @Las Vegas City Hall https://www. instagram.com/p/BZw8t4Hh-ps/";;@Las;#vegasstrong;"915002602978516992";htt

In [182]:
#before_filename = 'data_Las_Vegas/2017-09-25.csv'
# during_filename = 'data_Las_Vegas/2017-10-02.csv'
# after_filename = 'data_Las_Vegas/2017-10-20.csv'
train_bin_size = 21
test_bin_size = 3

vegas_directory = 'data_Las_Vegas'
houston_directory = 'data_Houston'
san_juan_directory = 'data_San_Juan'

In [5]:
#before = pd.read_csv(before_filename, delimiter=';', error_bad_lines=False)
#during = pd.read_csv(during_filename, delimiter=';', error_bad_lines=False)
#after = pd.read_csv(after_filename, delimiter=';', error_bad_lines=False)

Skipping line 1755: expected 10 fields, saw 11



In [184]:
def get_train_and_tests(directory, train_size=21, test_size=3):
    allFiles = sorted(glob.glob(directory + "/*.csv"))
    train_frame = pd.DataFrame()
    train_list = []
    tests_list = []
    test_list = []
    for index, file_ in enumerate(allFiles):
        df = pd.read_csv(file_,index_col=None, header=0, delimiter=';', error_bad_lines=False)
        if index < train_bin_size:
            #print "Adding {0} to training with {1} lines.".format(file_, len(df))
            train_list.append(df)
        elif (index + 1 - train_bin_size) % test_bin_size == 0:
            #print "Adding {0} to testing with {1} lines.".format(file_, len(df))
            test_list.append(df)
            test_set = pd.concat(test_list)
            #print "Adding a test set to training with {0} lines.".format(len(test_set))
            tests_list.append(test_set)
            test_list=[]
        else:
            #print "Adding {0} to testing with {1} lines.".format(file_, len(df))
            test_list.append(df)

    train_frame = pd.concat(list_)
    return (train_frame,tests_list)

In [185]:
vegas_train, vegas_tests = get_train_and_tests('data_Las_Vegas')

Skipping line 1755: expected 10 fields, saw 11

Skipping line 239: expected 10 fields, saw 11
Skipping line 1644: expected 10 fields, saw 11

Skipping line 296: expected 10 fields, saw 11



## Quick look at the data

In [6]:
before.head()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,NieveMalandra,2017-09-25 16:59,0,0,Beautiful Lace Top! Only 2 left! @Stardust Sty...,NaN,@Stardust,NaN,912466613185732609,https://twitter.com/NieveMalandra/status/91246...
1,Letzgitit,2017-09-25 16:56,0,0,Hip Hop Party Tonight with Me 9/25 @djvice at ...,NaN,@djvice @marqueelv,#marqueelv,912465816578412544,https://twitter.com/Letzgitit/status/912465816...
2,MSGDelk,2017-09-25 16:55,0,0,Little ones had picture day today.. Want to gi...,NaN,NaN,NaN,912465734105567232,https://twitter.com/MSGDelk/status/91246573410...
3,Letzgitit,2017-09-25 16:55,0,0,Party Tonight with Me 9/25 @J_Credible at @jew...,NaN,@J_Credible @jewellasvegas,#jewelnightclub,912465576018288641,https://twitter.com/Letzgitit/status/912465576...
4,riverdeepstyles,2017-09-25 16:54,0,0,Meat lover. @Shake Shack https://www. instagra...,NaN,@Shake,NaN,912465253245612032,https://twitter.com/riverdeepstyles/status/912...


In [7]:
during.head()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,fxm20193,2017-10-02 16:59,0,0,"Pray for vegas #prayforvegas #肉肉美國行 @The Linq,...",NaN,@The,#prayforvegas #,915003391880912901,https://twitter.com/fxm20193/status/9150033918...
1,krichardsonwi,2017-10-02 16:58,0,0,"I'm at @GoldSpikeDtlv in Las Vegas, NV https:/...",NaN,@GoldSpikeDtlv,NaN,915003111827271680,https://twitter.com/krichardsonwi/status/91500...
2,RalphPaglia,2017-10-02 16:58,0,0,Don’t let Steve Paddock win... visit #LasVegas...,NaN,NaN,#LasVegas,915003023180496896,https://twitter.com/RalphPaglia/status/9150030...
3,JLangelerNews,2017-10-02 16:56,1,2,Vigil at city hall #vegasstrong @Las Vegas Cit...,NaN,@Las,#vegasstrong,915002602978516992,https://twitter.com/JLangelerNews/status/91500...
4,libbybekins,2017-10-02 16:56,0,0,"my heart is heavy for this amazing city, stay ...",NaN,NaN,NaN,915002594044588033,https://twitter.com/libbybekins/status/9150025...


In [8]:
after.head()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,CnrgEvents,2017-10-20 16:59,0,0,#Repost @djquira (get_repost) ・・・ radiuspool a...,NaN,@djquira,#Repost #Stratosphere #CNRGEvents,921526326913261568,https://twitter.com/CnrgEvents/status/92152632...
1,ohsosexytash,2017-10-20 16:59,0,1,It was so nice in there #vegasfun #eiffeltower...,NaN,@Paris,#vegasfun #eiffeltower,921526256918847489,https://twitter.com/ohsosexytash/status/921526...
2,awmyposh,2017-10-20 16:58,0,3,I just fell in love with @gramercyvegas . . . ...,NaN,@gramercyvegas,NaN,921526087213113344,https://twitter.com/awmyposh/status/9215260872...
3,robynyates,2017-10-20 16:57,0,0,#work #letsgoplay @Windermere Prestige Propert...,NaN,@Windermere,#work #letsgoplay,921525934351458305,https://twitter.com/robynyates/status/92152593...
4,GoldenWestMgmt,2017-10-20 16:57,0,0,Just Listed in #LasVegas #NV . 9032 Square Kno...,NaN,NaN,#LasVegas #NV,921525733322665984,https://twitter.com/GoldenWestMgmt/status/9215...


## Hashtags
Let's confirm we're in the right timeframe by looking at the most popular hashtag sets.

In [9]:
before['count'] = before.groupby('hashtags')['hashtags'].transform(pd.Series.value_counts)
before.sort('count', ascending=False)
before.hashtags.dropna().head()

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


1                  #marqueelv
3             #jewelnightclub
5     #OneDO #DODoesDiversity
7        #lifeisbeautiful2017
11                  #chamafio
Name: hashtags, dtype: object

In [10]:
during['count'] = during.groupby('hashtags')['hashtags'].transform(pd.Series.value_counts)
during.sort('count', ascending=False)
during.hashtags.dropna().head()

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


0     #prayforvegas #
2           #LasVegas
3        #vegasstrong
5              #Vegas
9    #PrayForLasVegas
Name: hashtags, dtype: object

In [11]:
after['count'] = after.groupby('hashtags')['hashtags'].transform(pd.Series.value_counts)
after.sort('count', ascending=False)
after.hashtags.dropna().head()

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


0    #Repost #Stratosphere #CNRGEvents
1               #vegasfun #eiffeltower
3                    #work #letsgoplay
4                        #LasVegas #NV
5                               #theUN
Name: hashtags, dtype: object

## Word Counts

In [12]:
other_stopwords = ['http://', 'https://www.', '\xe2\x80\xa6']
stop_words = set(stopwords.words('english'))
stop_words |= set(other_stopwords)

In [13]:
results = Counter()
before['text'].str.lower().str.split().apply(results.update)
for w in stop_words:
    del results[w]
results.most_common()

[('las', 83),
 ('vegas', 78),
 ('beautiful', 58),
 ('vegas,', 56),
 ('@las', 34),
 ('@life', 34),
 ('-', 31),
 ("i'm", 28),
 ('nevada', 28),
 ('#lasvegas', 27),
 ('.', 27),
 ('bit.ly/xxmewb', 26),
 ('accident', 26),
 ('nv', 26),
 ('#vegastraffic', 26),
 ('&', 24),
 ('store', 19),
 ('posted', 19),
 ('new', 17),
 ('!', 17),
 ('life', 16),
 ('day', 16),
 ('photo', 15),
 ('casino', 15),
 ('strip', 14),
 ('@urban', 14),
 ('like', 14),
 ('got', 13),
 ('#theun', 13),
 ('#vegas', 13),
 ('#lifeisbeautiful', 13),
 ('necessities', 13),
 ('rd', 12),
 ('happy', 12),
 ('love', 11),
 ('->', 11),
 ('great', 11),
 ('blvd', 11),
 ('wind', 10),
 ('size', 10),
 ('hotel', 10),
 ('nv)', 10),
 ('nb', 9),
 ('time', 9),
 ('one', 9),
 ('15', 9),
 ('get', 9),
 ('last', 9),
 ('the\xe2\x80\xa6', 8),
 ('amazing', 8),
 ('weekend', 8),
 ('birthday', 8),
 ('2', 8),
 ('--', 8),
 ('see', 8),
 ("it's", 8),
 ('party', 8),
 ('preowned', 8),
 ('reported', 7),
 ('phone', 7),
 ('16%', 7),
 ('right', 7),
 ('thank', 7),
 ('show

In [21]:
results = Counter()
during['text'].str.lower().str.split().apply(results.update)
during_v = len(results)
for w in stop_words:
    del results[w]
results.most_common()

[('vegas', 465),
 ('las', 423),
 ('vegas,', 317),
 ('@las', 309),
 ('nevada', 254),
 ('#lasvegas', 219),
 ('prayers', 161),
 ('#prayforvegas', 125),
 ('last', 119),
 ('heart', 110),
 ('#prayforlasvegas', 109),
 ('&', 106),
 ('thoughts', 96),
 ('strip', 96),
 ('pray', 95),
 ('-', 93),
 ('love', 91),
 ('people', 83),
 ('.', 82),
 ('affected', 77),
 ("i'm", 75),
 ('victims', 75),
 ('en', 72),
 ('go', 71),
 ('#vegas', 71),
 ('everyone', 68),
 ('the\xe2\x80\xa6', 67),
 ('praying', 65),
 ('vegas.', 64),
 ('please', 62),
 ('nv', 60),
 ('city', 58),
 ('friends', 57),
 ('today', 54),
 ('posted', 53),
 ('blood', 52),
 ('one', 51),
 ('families', 51),
 ('photo', 50),
 ('night', 49),
 ('casino', 47),
 ('bay', 47),
 ('#repost', 46),
 ('family', 46),
 ('thank', 46),
 ('tragedy', 44),
 ('us', 43),
 ('and\xe2\x80\xa6', 43),
 ("it's", 43),
 ('goes', 43),
 ('happened', 43),
 ('time', 42),
 ('to\xe2\x80\xa6', 40),
 ('many', 39),
 ('sad', 38),
 ('world', 38),
 ('de', 38),
 ('shooting', 38),
 ('going', 36),

In [15]:
results = Counter()
after['text'].str.lower().str.split().apply(results.update)
for w in stop_words:
    del results[w]
results.most_common()

[('vegas', 84),
 ('las', 82),
 ('-', 46),
 ('#lasvegas', 44),
 ('.', 38),
 ('vegas,', 37),
 ('@las', 28),
 ('&', 28),
 ('accident', 24),
 ('tonight', 24),
 ("i'm", 23),
 ('#vegastraffic', 23),
 ('nv', 23),
 ('bit.ly/xxmewb', 22),
 ('nevada', 22),
 ('casino', 22),
 ('max', 19),
 ('#vegas', 19),
 ('#adobemax', 18),
 ('strip', 17),
 ('@adobe', 16),
 ('resort', 16),
 ('nightclub', 16),
 ('great', 15),
 ('time', 14),
 ('vip', 14),
 ('friday', 14),
 ('@\xe2\x80\xa6', 14),
 ('posted', 13),
 ('new', 12),
 ('photo', 12),
 ('check', 12),
 ('us', 12),
 ('drinking', 12),
 ('weekend', 11),
 ('#repost', 11),
 ('#vegasfun', 11),
 ('hotel', 11),
 ('see', 11),
 ('back', 11),
 ('blvd', 11),
 ('text', 11),
 ('nv)', 11),
 ('\xe2\x80\x94', 11),
 ("marino's", 10),
 ('bday', 10),
 ('adobe', 10),
 ('divas', 10),
 ('night', 10),
 ('today', 10),
 ('@frank', 10),
 ('come', 10),
 ('online.', 10),
 ('store', 10),
 ('party', 10),
 ('chateau', 9),
 ('769-233-3161', 9),
 ('hollywood', 9),
 ('#theun', 9),
 ('internati

## NGram Model

In [86]:
def get_train_test_sents(corpus, split=0.8, shuffle=True):
    """Get train and test sentences.
    Args:
      corpus: nltk.corpus that supports sents() function
      split (double): fraction to use as training set
      shuffle (int or bool): seed for shuffle of input data, or False to just
      take the training data as the first xx% contiguously.
    Returns:
      train_sentences, test_sentences ( list(list(string)) ): the train and test
      splits
    """
    sentences = np.array(corpus, dtype=object)
    fmt = (len(sentences), sum(map(len, sentences)))
    print "Loaded %d sentences (%g tokens)" % fmt

    if shuffle:
        rng = np.random.RandomState(shuffle)
        rng.shuffle(sentences)  # in-place
    train_frac = 0.8
    split_idx = int(train_frac * len(sentences))
    train_sentences = sentences[:split_idx]
    test_sentences = sentences[split_idx:]

    fmt = (len(train_sentences), sum(map(len, train_sentences)))
    print "Training set: %d sentences (%d tokens)" % fmt
    fmt = (len(test_sentences), sum(map(len, test_sentences)))
    print "Test set: %d sentences (%d tokens)" % fmt

    return train_sentences, test_sentences

In [99]:
#train_sents, test_sents = get_train_test_sents(during['text'].str.lower().str.split(), split=0.8, shuffle=False)
#Remove stop words
train_sents, test_sents = get_train_test_sents(during['text'].str.lower().str.split().apply(lambda x: [item for item in x if item not in stop_words])
, split=0.8, shuffle=False)
vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(train_sents)), size=during_v)
# vocab = vocabulary.Vocabulary((utils.canonicalize_word(w) for w in utils.flatten(corpus.sents())), size=V)
print "Train set vocabulary: %d words" % vocab.size

Loaded 1999 sentences (19724 tokens)
Training set: 1599 sentences (15876 tokens)
Test set: 400 sentences (3848 tokens)
Train set vocabulary: 6462 words


In [100]:
print train_sents[0]
print during_v
print vocab.id_to_word

['pray', 'vegas', '#prayforvegas', '#\xe8\x82\x89\xe8\x82\x89\xe7\xbe\x8e\xe5\x9c\x8b\xe8\xa1\x8c', '@the', 'linq,', 'las', 'vegas', 'highest\xe2\x80\xa6', 'instagram.com/p/bzw9oykb9w_r', 'ge0ai4sqfydfx5yjm28xsaabuu0/']
7750
{0: '<s>', 1: '</s>', 2: '<unk>', 3: 'vegas', 4: 'las', 5: 'vegas,', 6: '@las', 7: 'nevada', 8: '#lasvegas', 9: 'prayers', 10: 'last', 11: '#prayforvegas', 12: 'heart', 13: '-', 14: '#prayforlasvegas', 15: 'love', 16: '&', 17: '.', 18: 'strip', 19: 'people', 20: 'thoughts', 21: 'pray', 22: "i'm", 23: 'affected', 24: 'victims', 25: 'go', 26: '#vegas', 27: 'vegas.', 28: 'the\xe2\x80\xa6', 29: 'DGDG', 30: 'everyone', 31: 'nv', 32: 'city', 33: 'please', 34: 'en', 35: 'praying', 36: 'one', 37: 'night', 38: 'friends', 39: 'blood', 40: '#repost', 41: 'today', 42: 'and\xe2\x80\xa6', 43: 'tragedy', 44: 'casino', 45: 'families', 46: 'thank', 47: 'bay', 48: 'DG', 49: 'to\xe2\x80\xa6', 50: 'happened', 51: 'posted', 52: 'us', 53: 'time', 54: 'family', 55: 'goes', 56: "it's", 57

In [101]:
def sents_to_tokens(sents):
    """Returns an flattened list of the words in the sentences, with padding for a trigram model."""
    padded_sentences = (["<s>", "<s>"] + s + ["</s>"] for s in sents)
    # This will canonicalize words, and replace anything not in vocab with <unk>
    return np.array([utils.canonicalize_word(w, wordset=vocab.wordset) 
                     for w in utils.flatten(padded_sentences)], dtype=object)

train_tokens = sents_to_tokens(train_sents)
test_tokens = sents_to_tokens(test_sents)
print "Sample data: \n" + repr(train_tokens[:20])

Sample data: 
array(['<s>', '<s>', 'pray', 'vegas', '#prayforvegas',
       '#\xe8\x82\x89\xe8\x82\x89\xe7\xbe\x8e\xe5\x9c\x8b\xe8\xa1\x8c',
       '@the', 'linq,', 'las', 'vegas', 'highest\xe2\x80\xa6',
       'instagram.com/p/bzw9oykb9w_r', 'ge0ai4sqfydfx5yjm28xsaabuu0/',
       '</s>', '<s>', '<s>', "i'm", '@goldspikedtlv', 'las', 'vegas,'], dtype=object)


In [93]:
import ngram_lm
import ngram_utils
reload(ngram_lm)

# Uncomment the line below for the model you want to run.
Model = ngram_lm.AddKTrigramLM
#Model = ngram_lm.KNTrigramLM

t0 = time.time()
print "Building trigram LM...",
lm = Model(train_tokens)
print "done in %.02f s" % (time.time() - t0)
ngram_utils.print_stats(lm)

Building trigram LM... done in 0.28 s
=== N-gram Language Model stats ===
0 unique 1-grams
0 unique 2-grams
22614 unique 3-grams
Optimal memory usage (counts only): 0 MB


In [102]:
lm.set_live_params(k = 0.001, delta=0.75)

In [103]:
max_length = 20
num_sentences = 5

for _ in range(num_sentences):
    seq = ["<s>", "<s>"]
    for i in range(max_length):
        seq.append(ngram_utils.predict_next(lm, seq))
        # Stop at end-of-sentence.
        if seq[-1] == "</s>": break
    print " ".join(seq)
    print "[{1:d} tokens; log P(seq): {0:.02f}]".format(*ngram_utils.score_seq(lm, seq))
    print ""

<s> <s> having just kind. 3thxeq3ty7w-xtio1ltsj4afue0/ limo, | forget! day, instagram.com/p/bzwpljafgst/ cars disasters info… #teachpeopleaboutpeople #swingeyewear eating love instagram.com/p/bzwk9ltns6q/ #heatercentral instagram.com/p/bzwqavslswp/ vibes.
[20 tokens; log P(seq): -241.25]

<s> <s> let tragic mass… instagram.com/p/bzwq3r4g0xtb silent schon inside von instagram.com/p/bzwacvbjkan/ recent instagram.com/p/bzwiqy-fpsg/" #leroychops listos assassinate j8thbbad_y3q87xwnfzhfn5roe0/ desde need. so, momento instagram.com/p/bzw1zvlfjhq/
[20 tokens; log P(seq): -251.02]

<s> <s> the last time vegas! animada, #musiclife devastated. instagram.com/p/bzw2w6mhm41/ planned seven crimes @wbke @diplo instagram.com/p/bzwlyabg1fs/ instagram.com/p/bzwu1ighlqe/ #thepoweroflove bothered photos any instagram.com/p/bzwm-sldb5o/
[20 tokens; log P(seq): -227.15]

<s> <s> taxi o_luggfgjpjzagg7zmigppdslo0/ shooting. @alexander instagram.com/p/bzwtujebiy3/ @teamnosleep earth- ready instagram.com/p/bzwv

In [104]:
log_p_data, num_real_tokens = ngram_utils.score_seq(lm, train_tokens)
print "Train perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens))

log_p_data, num_real_tokens = ngram_utils.score_seq(lm, test_tokens)
print "Test perplexity: %.02f" % (2**(-1*log_p_data/num_real_tokens))

Train perplexity: 603.64
Test perplexity: 4449.20
